# California CZU/SCU VIIRS Fire Map
This notebook uses data from the VIIRS satellites to plot detected fire events in Northern California over the time period of August 18th to August 28th, 2020. 

In order to obtain the data needed, visit the FIRMS Archive Download tool https://firms.modaps.eosdis.nasa.gov/download/create.php. Use the bounding box tool to select the appropriate area and export the data as csv. Select both VIIRS satellites as both will make separate passes over the area giving the most coverage.

In [17]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import shutil
from datetime import datetime
from pathlib import Path
from PIL import Image

In [45]:
def make_or_clear_path(path):
    if os.path.exists(path):
        shutil.rmtree(path)
        
    os.mkdir(path)
    
make_or_clear_path('czu_fire')
make_or_clear_path('scu_fire')

# Import Data
Convert the times to Pacific and use -122 longitude as a rough separation of SCU and CZU fires

In [30]:
def read_data(path):
    df = pd.read_csv(path, dtype={'acq_time': str})
    df['dt'] = pd.to_datetime(df['acq_date'] + ' ' + df['acq_time'])\
               .dt.tz_localize('UTC')\
               .dt.tz_convert('US/Pacific')\
               .dt.tz_localize(None)
    df = df[df['confidence'] != 'low']
    df['fire'] = df['longitude'].apply(lambda x: 'CZU' if x <= -122 else 'SCU')
    df['file'] = path
    return df

# Change report paths as necessary
df = pd.concat([
    read_data('fire_nrt_V1_148688.csv'),
    read_data('fire_nrt_J1V-C2_148687.csv'),
], axis=0)

df = df.sort_values(by='dt').set_index('dt')
df.head()

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,fire,file,brightness,bright_t31
dt,,,,,,,,,,,,,,,,,,
2020-08-16 02:36:00,37.95082,-122.39911,303.1,0.39,0.44,2020-08-16,0936,N,VIIRS,n,2.0NRT,288.2,1.7,N,CZU,fire_nrt_V1_148688.csv,NaN,NaN
2020-08-16 02:36:00,37.88188,-121.18634,307.4,0.51,0.41,2020-08-16,0936,N,VIIRS,n,2.0NRT,296.0,1.4,N,SCU,fire_nrt_V1_148688.csv,NaN,NaN
2020-08-16 02:36:00,37.94184,-122.39156,295.2,0.39,0.44,2020-08-16,0936,N,VIIRS,n,2.0NRT,280.6,0.7,N,CZU,fire_nrt_V1_148688.csv,NaN,NaN
2020-08-16 13:06:00,37.62751,-121.77472,NaN,0.51,0.66,2020-08-16,2006,1,VIIRS,n,2.0NRT,NaN,5.6,D,SCU,fire_nrt_J1V-C2_148687.csv,334.8,303.7
2020-08-16 13:06:00,37.49356,-121.42377,NaN,0.49,0.65,2020-08-16,2006,1,VIIRS,n,2.0NRT,NaN,14.6,D,SCU,fire_nrt_J1V-C2_148687.csv,330.4,297.4


# Resample Data
Resamples the data to a rolling 24 hour lookback. For every 6 hours between 8/16 and 8/28, filter the data for the last 24 hours. This adds a little more stability to the data for visualization.

In [33]:
def resample_data(df):
    new_df = pd.DataFrame()

    for i in pd.date_range(start='2020-08-16', end='2020-08-28', freq='6H'):
        st = i - pd.Timedelta(24, 'h')
        ed = i
        dff = df.loc[st:ed, ['latitude','longitude','frp','fire']].reset_index()
        dff['dt'] = i

        new_df = new_df.append(dff)

    new_df = new_df.sort_values(by='dt')
    new_df['dts'] = new_df['dt'].astype(str)
    new_df['seq'] = new_df['dt'].rank(method='dense')
    new_df = new_df.rename(columns={'frp': 'Fire Radiative Power (MW)'})
    
    return new_df

df_resampled = resample_data(df)

# Plot Fires
Plot fires using Plotly's density plot. The dark background requires a free Mapbox API token which is stored in a file named `token`. Outputs each fire to it's own folder.

In [46]:
def plot_fire(data, fire_name, output_path):
    with open('token', 'r') as f:
        token = f.read()

    df = data.loc[data['fire'] == fire_name]
    frp_upper = df['Fire Radiative Power (MW)'].quantile(0.80)
    center = {'lat': df['latitude'].mean(), 'lon': df['longitude'].mean()}
    
    for i, idx in enumerate(df.dt.unique()):
        ft = datetime.strptime(str(idx)[:16], '%Y-%m-%dT%H:%M').strftime('%b %d, %Y - %I:%M %p PDT')
        df_subset = df[df['dt'] == idx]
        fig = px.density_mapbox(df_subset, lat="latitude", lon="longitude", z='Fire Radiative Power (MW)',
                                center=center,
                                radius=5,
                                range_color=(0, frp_upper),
                                color_continuous_scale=px.colors.sequential.YlOrRd,
                                zoom=9, height=700, width=1000)
        fig.update_layout(
            mapbox_style="dark",
            mapbox_accesstoken=token,
            margin={'r':0,'l':0,'b':0},
            title={
                'text': f'{fire_name} Fire - VIIRS 350m Satellite Image - 24H Period - {ft}',
                'xanchor': 'center',
                'x': 0.4,
                'font': {'size': 16}
            },
            font={
                'family': 'Helvetica'
            }
        )
        fig.write_image(f'{output_path}/{i}.png')
        
plot_fire(df_resampled, 'CZU', 'czu_fire')
plot_fire(df_resampled, 'SCU', 'scu_fire')

# Make GIFs
Loop through each folder and combine into gifs

In [47]:
def make_gif(path, output):
    p = Path(path)
    paths = sorted(p.glob('*.png'), key=lambda x: int(os.path.splitext(x.name)[0]))
    images = []
    
    for img_path in paths:
        images.append(Image.open(img_path))
        
    images[0].save(output,
                   append_images=images[1:],
                   save_all=True,
                   duration=500,
                   loop=0)
    
make_gif('czu_fire', 'czu_fire.gif')
make_gif('scu_fire', 'scu_fire.gif')

# CZU Fire
![CZU](czu_fire.gif)

# SCU Fire
![SCU](scu_fire.gif)